# Crypto Challenge Set 2

## 9. Implement PKCS#7 padding

https://cryptopals.com/sets/2/challenges/9

In [1]:
def pkcs7(block,blocksize,pad=b'\x04'):
    return block+max(0,(blocksize-len(block)))*pad

In [2]:
block = b"YELLOW SUBMARINE"
pkcs7(block,20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## 10. Implement CBC mode

https://cryptopals.com/sets/2/challenges/10

https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Cipher_block_chaining_(CBC)

In [3]:
from Crypto.Cipher import AES

def fixedXOR(a,b):
    return bytes([an^bn for an,bn in zip(a,b)])

def AES_CBC_decrypt(cipher,key):
    aes = AES.new(key, AES.MODE_ECB)
    bsize = len(key)
    IV = bsize*b"\x00"
    blocks = [ cipher[i:i+bsize] for i in range(0,len(cipher),bsize) ]
    plaintext = ""
    for i in range(len(blocks)):
        # decrypt block with AES ECB mode
        dec = aes.decrypt(blocks[i])
        # XOR with IV or previous cipher block
        dec = fixedXOR(dec,IV) if i==0 else fixedXOR(dec,blocks[i-1])
        plaintext += "".join([chr(c) for c in dec])
    return plaintext

In [4]:
from binascii import a2b_base64

key = b"YELLOW SUBMARINE"

with open("input/10.txt") as f:
    cipher = a2b_base64(f.read().replace("\n",""))
    plaintext = AES_CBC_decrypt(cipher,key)
    print(plaintext[0:241])
    
with open('input/plaintext.txt', 'w') as f:
    f.write(plaintext)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy


In [5]:
import binascii

def AES_CBC_encrypt(plaintext,key,IV=None):
    aes = AES.new(key, AES.MODE_ECB)
    bsize = len(key)
    if IV==None:
        IV = bsize*b"\x00"
    plainb = bytes(plaintext.encode()) # convert plaintext to bytes
    if len(plainb)%bsize: # padding to multiple of block size if needed 
        plainb = pkcs7(plainb,len(plainb)+bsize-len(plainb)%bsize)
    blocks = [ plainb[i:i+bsize] for i in range(0,len(plainb),bsize) ]
    ciphbl = []
    cipher = b""
    for i in range(len(blocks)):
        # XOR with IV or previous cipher block
        b = fixedXOR(blocks[i],IV) if i==0 else fixedXOR(blocks[i],ciphbl[i-1])
        # encrypt IVed block with AES ECB mode
        enc = aes.encrypt(b)
        ciphbl.append(enc)
        cipher += enc
    return cipher

In [6]:
cipher = AES_CBC_encrypt(plaintext,key)
plaintext = AES_CBC_decrypt(cipher,key)
print(plaintext[0:241])

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy


## 11. An ECB/CBC detection oracle

https://cryptopals.com/sets/2/challenges/11

> Write a function to generate a random AES key; that's just 16 random bytes. `os.urandom()` return a string of size random bytes suitable for cryptographic use.

https://docs.python.org/3.8/library/os.html#os.urandom

In [30]:
import os

def randAESkey(keylen=16):
    return os.urandom(keylen)

randAESkey()

b'C\x81\xfbuY\xb9,W4m]H,\xe3C\x95'

> Write a function that encrypts data under an unknown key, e.g. a function that generates a random key and encrypts under it. Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext. Have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). 

> Detect the block cipher mode the function is using each time. 

In [31]:
import os
import random
from Crypto.Cipher import AES

def encryption_oracle(plaintext):
    # generate a 16-bytes random key
    keysize = 16
    key = os.urandom(keysize)
    plainb = bytes(plaintext.encode()) # plaintext in bytes
    # prepend and append bytes
    plainb = os.urandom(random.randint(5,10))+plainb+os.urandom(random.randint(5,10)) 
    # pad the plaintext to a multiple of keysize
    if len(plainb)%keysize:
        plainb = pkcs7(plainb,len(plainb)+keysize-len(plainb)%keysize)
    cipher = ""
    mode = random.randint(0,1)
    if mode==1: # encrypt under ECB
        aes_ecb = AES.new(key, AES.MODE_ECB)
        cipher = aes_ecb.encrypt(plainb)
    else: # encrypt under CBC
        IV = os.urandom(keysize)
        aes_cbc = AES.new(key, AES.MODE_CBC, IV) 
        cipher = aes_cbc.encrypt(plainb)
    return mode, cipher

In [33]:
def isAESECB(cipher,blocksize=16):
    blocks = [ cipher[i:i+blocksize] for i in range(0,len(cipher),blocksize) ]
    return bool(len(blocks) - len(set(blocks)))

In [40]:
ntot = 10

# This is the poem provided as ciphertext at challenge 10. No problem in guessing the cipher mode with it!
with open("input/plaintext.txt") as f:
    plaintext = f.read().strip("\n")

# This plaintext is too short and with no repetitions, it's impossible to detect ECB mode! 
#plaintext = "the quick brown fox jumps over the lazy dog"

# This is an abtract of an ATLAS paper (a real life text!): 
# it's relatively long, with some repetitions (but not many). Detection does not work :-(
#with open("input/HIGG-2018-51_abstract.txt") as f:
#    plaintext = f.read().strip("\n")

for _ in range(ntot):
    # mode is the true encryption used (0 = CBC, 1 = ECB)
    mode, cipher = encryption_oracle(plaintext) 
    pred = isAESECB(cipher)
    print(mode,pred)

0 False
1 True
0 False
0 False
0 False
0 False
1 True
0 False
0 False
0 False


## 12. Byte-at-a-time ECB decryption (Simple)

https://cryptopals.com/sets/2/challenges/12

In [42]:
import binascii
import os
import random
from Crypto.Cipher import AES

def AES_ECB_encrypt(plainb,key):
    # pad plaintext to multiple of keysize if needed
    keysize = len(key)
    if len(plainb)%keysize: 
        plainb = pkcs7(plainb,len(plainb)+keysize-len(plainb)%keysize)
    # encode
    aes_ecb = AES.new(key,AES.MODE_ECB)
    return aes_ecb.encrypt(plainb)

def AES_ECB_decrypt(cipher,key):
    aes_ecb = AES.new(key,AES.MODE_ECB)
    return aes_ecb.decrypt(cipher)

class Oracle1():
    def __init__(self):
        self.key = os.urandom(16)
        self.unknown = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
        self.unknown_bin = binascii.a2b_base64(self.unknown)
        self.unknown_str = self.unknown_bin.decode()
    
    def encrypt(self,string=""):
        '''Encrypt unknown string appending the injection string'''
        return AES_ECB_encrypt(string.encode()+self.unknown_bin,self.key)
    
    def decrypt(self,string):
        return AES_ECB_decrypt(string,self.key).decode()

### Discover the block size of the cipher

In [43]:
def guessSize(oracle):
    my_string = ""
    cipher0 = oracle.encrypt(my_string)
    cipher=cipher0
    while len(cipher0)==len(cipher):
        my_string += "A"
        cipher = oracle.encrypt(my_string)
    return len(cipher)-len(cipher0)

In [48]:
oracle1 = Oracle1()

bsize = guessSize(oracle1)
print("Block size =",bsize)

Block size = 16


### Detect that the function is using ECB

Feed the oracle function with a repeating string long enough to ensure that repetitions would be visible regardless of the content of the unknown string.

In [49]:
my_string = 2*bsize*"A"
print("Is the function using ECB?", isAESECB(oracle1.encrypt(my_string)))

Is the function using ECB? True


### Attack the oracle with injection string of varying lenght

In [54]:
def findPlainTextLenght(oracle):
    bsize = guessSize(oracle)
    i = 0
    cipher_0 = oracle1.encrypt("")
    cipher_i = cipher_0
    for i in range(bsize):
        cipher_i = oracle.encrypt(i*"A")
        if len(cipher_0)<len(cipher_i):
            return len(cipher_0)-i+1

def byte_at_a_time_ECB_decryption(oracle):
    injsize = len( oracle1.encrypt(""))
    plaintext = ""
    for k in range(injsize):
        my_string = (injsize-1-len(plaintext))*"A"
        cipher_0 = oracle1.encrypt(my_string)
        for i in range(256):
            cipher_i = oracle1.encrypt(my_string+plaintext+chr(i))
            if cipher_0[:injsize] == cipher_i[:injsize]:
                plaintext += chr(i)
                break
    return plaintext[:findPlainTextLenght(oracle)]

In [55]:
decrypt_str = byte_at_a_time_ECB_decryption(oracle1)

assert(decrypt_str==oracle1.unknown_str)

print(decrypt_str)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



## 13. ECB cut-and-paste

https://cryptopals.com/sets/2/challenges/13

In [65]:
class Profile:
    def __init__(self):
        self.key = os.urandom(16)
        
    def parse(self,string):
        '''k=v parsing routine'''
        return { i.split("=")[0]: i.split("=")[1] for i in string.split("&") }
    
    def profile_for(self,email):
        # Your "profile_for" function should not allow encoding metacharacters (& and =). 
        if b"&" in email.encode() or b"=" in email.encode():
            #raise ValueError("Invalid email address")
            print("Invalid email address")
            return b""
        return b"email=" + email.encode() + b"&uid=10&role=user"
    
    def encrypt(self,email):
        profile = self.profile_for(email)
        if profile != "":
            return AES_ECB_encrypt(profile,key)

    def decrypt(self,encprof):
        return AES_ECB_decrypt(encprof,key).decode().strip("\x04")

In [66]:
profile = Profile()

print(profile.parse("foo=bar&baz=qux&zap=zazzle"))

print(profile.profile_for("foo@bar.com"))

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}
b'email=foo@bar.com&uid=10&role=user'


In [67]:
encprof = profile.encrypt("foo@bar.com")
print(profile.decrypt(encprof))

email=foo@bar.com&uid=10&role=user


### Attack

In [73]:
def splitBlocks(string,bsize):
    return [ string[i:i+bsize] for i in range(0,len(string),bsize) ]

In [80]:
# Guessing the block size
bsize = guessSize(profile)

# Forging a fake email to get the 'user' role at the beginning of a block. 
# Since blockas are encoded separately I could then replace that (encoded) 
# 'user' block only with the (encoded) 'admin' role

attack_email = "foo@bar.com"
attack_target = b'user'

while True:
    prof = profile.profile_for(attack_email)
    if splitBlocks(prof,bsize)[-1] == attack_target:
        print("Attack email:",attack_email)
        break
    attack_email = "f"+attack_email
    
print(splitBlocks(prof,bsize))

Attack email: fffoo@bar.com
[b'email=fffoo@bar.', b'com&uid=10&role=', b'user']


In [81]:
cipher_user = profile.encrypt(attack_email)
print(cipher_user)
print(len(cipher_user))

b',+J$\x82\xbc\xd6\x11\xcfD8\x9c\x1cp\x07!J}6D?\xb6a\xc0G\x97f_\x9e\xc7\xc2\x0c\xe5p\x8d:\xb1\xb5\x0f\xc8V\x01m!\xd6\xb7~\x83'
48


In [84]:
attack_role = b'admin'

# padding attack role to fill one block

attack_block = pkcs7(attack_role,bsize)

# devoting two blocks to forged email, so that attack role (admin)
# will be at tbe beginning of third block

prepend = "email="
attack_email_2 = (bsize-len(prepend))*b"f"+(bsize-len(b"foo@bar.com"))*b"f"+b"foo@bar.com"
prof_attack = profile.profile_for( (attack_email_2+attack_block).decode() )
print(splitBlocks(prof_attack,bsize))

[b'email=ffffffffff', b'ffffffoo@bar.com', b'admin\x04\x04\x04\x04\x04\x04\x04\x04\x04\x04\x04', b'&uid=10&role=use', b'r']


In [87]:
# Getting the cipher for the forged email address

encprof_attackrole = profile.encrypt( (attack_email_2+attack_block).decode() )

splitBlocks(encprof_attackrole,bsize)[2]

b'\xc3\xd9\xcb\xd2\xe2\x89\xa9N\xde\n\x98\x91)\x82"K'

In [89]:
# merging blocks from forged 'user' profile with block from admin attack

encprof_attack = b"".join(splitBlocks(cipher_user,bsize)[:2]+[splitBlocks(encprof_attackrole,bsize)[2]] )
encprof_attack

b',+J$\x82\xbc\xd6\x11\xcfD8\x9c\x1cp\x07!J}6D?\xb6a\xc0G\x97f_\x9e\xc7\xc2\x0c\xc3\xd9\xcb\xd2\xe2\x89\xa9N\xde\n\x98\x91)\x82"K'

In [91]:
profile.decrypt(encprof_attack)

'email=fffoo@bar.com&uid=10&role=admin'